### Проект: вариант 3
Представьте, что вы работаете в крупном дейтинговом приложении.

Помимо базовых функций, в приложении также имеется премиум-подписка, которая дает доступ к ряду важных дополнительных возможностей. Был проведен A/B тест, в рамках которого для новых пользователей из нескольких стран была изменена стоимость премиум-подписки* при покупке через две новые платежные системы. При этом стоимость пробного периода оставалась прежней.

Проверьте:

Был ли эксперимент успешен в целом.
Проанализируйте, имеет ли нововведение смысл среди каких-либо конкретных групп пользователей.

*Деньги за подписку списываются ежемесячно до тех пор, пока пользователь её не отменит.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from scipy.stats import mannwhitneyu, f_oneway,kruskal, levene, ttest_ind,norm

from tqdm.auto import tqdm

In [2]:
# Загружаем данные
transactions_test      = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_transactions_test.csv', sep = ';')
users_test             = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_users_test.csv', sep = ';')
transactions_control_1 = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_transactions_control_1.csv', sep = ';')
users_control_1        = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_users_control_1.csv', sep = ';')
transactions_control_2 = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_transactions_control_2.csv', sep = ';')
users_control_2        = pd.read_csv('/home/jupyter-v.panshina-15/final_project/Проект_3_users_control_2.csv', sep = ';')

In [3]:
# Меняем формат даты
transactions_test['joined_at']      = pd.to_datetime(transactions_test['joined_at'])
transactions_test['paid_at']        = pd.to_datetime(transactions_test['paid_at'])
transactions_control_1['joined_at'] = pd.to_datetime(transactions_control_1['joined_at'])
transactions_control_1['paid_at']   = pd.to_datetime(transactions_control_1['paid_at'])
transactions_control_2['joined_at'] = pd.to_datetime(transactions_control_2['joined_at'])
transactions_control_2['paid_at']   = pd.to_datetime(transactions_control_2['paid_at'])

In [4]:
# Проверяем корректность данных
transactions_test.dtypes

uid                      int64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                  int64
payment_id               int64
from_page               object
product_type            object
dtype: object

In [5]:
transactions_control_1.dtypes

uid                    float64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                float64
payment_id             float64
from_page               object
product_type            object
dtype: object

In [6]:
transactions_control_2.dtypes

uid                      int64
country                 object
joined_at       datetime64[ns]
paid_at         datetime64[ns]
revenue                  int64
payment_id               int64
from_page               object
product_type            object
dtype: object

In [7]:
# Проверяем на пустые значения
transactions_test.isna().sum()

uid             0
country         0
joined_at       0
paid_at         0
revenue         0
payment_id      0
from_page       0
product_type    0
dtype: int64

In [8]:
transactions_control_1.isna().sum()

uid             630
country         630
joined_at       630
paid_at         630
revenue         630
payment_id      630
from_page       630
product_type    630
dtype: int64

In [9]:
transactions_control_2.isna().sum()

uid             0
country         0
joined_at       0
paid_at         0
revenue         0
payment_id      0
from_page       0
product_type    0
dtype: int64

In [10]:
#Удаляем в transactions_control_1 пустые значения
transactions_control_1 = transactions_control_1.dropna()

In [11]:
# Проверяем дубликаты
transactions_test.drop_duplicates().uid.count()
transactions_test.uid.count()

273

In [12]:
users_test.drop_duplicates().uid.count()
users_test.uid.count()

4308

In [13]:
# Дропаем дубликаты и обновляем датафреймы
transactions_test      = transactions_test.drop_duplicates()
transactions_control_1 = transactions_control_1.drop_duplicates()
transactions_control_2 = transactions_control_2.drop_duplicates()

#### Объеденяем данные:

In [14]:
test_df      = users_test.merge(transactions_test, how='left', on='uid', suffixes=('_reg', '_trn'))
control_1_df = users_control_1.merge(transactions_control_1, how='left', on='uid', suffixes=('_reg', '_trn'))
control_2_df = users_control_2.merge(transactions_control_2, how='left', on='uid', suffixes=('_reg', '_trn'))

#### Проверяем на ошибки:

In [15]:
test_df.uid.count() == transactions_test.uid.count()

False

In [16]:
test_df.revenue.sum() == transactions_test.revenue.sum()

True

In [17]:
control_1_df.uid.count() == transactions_control_1.uid.count()

False

In [18]:
control_1_df.revenue.sum() == transactions_control_1.revenue.sum()

True

In [19]:
control_2_df.uid.count() == transactions_control_2.uid.count()

False

In [20]:
control_2_df.revenue.sum() == transactions_control_2.revenue.sum()

True

#### Проверим был ли эксперимент успешен в целом.
Проверим(оценка значимого изменения распределений через Манна — Уитни и изменения средних через Bootstrap):
- ARPU на клиента;
- ARPPU на клиента (test vs controls) - выросло ли значимо;
- частота покупок на клиента - упала или осталась на том же уровне.


Анализировать будем по разрезам ниже: 

- gender
- was_premium
- is_premium
- product_type

#### Найдем p-value, будем сравнивать группы в разных тестах.

- ARPU на клиента;

In [21]:
test= users_test.total_revenue
c_1 = users_control_1.total_revenue 
c_2 = users_control_2.total_revenue

In [22]:
mannwhitneyu(c_1,c_2)

MannwhitneyuResult(statistic=9257464.0, pvalue=0.9108956670341875)

In [23]:
mannwhitneyu(test,c_1)

MannwhitneyuResult(statistic=9254877.5, pvalue=0.016463749472916307)

In [24]:
mannwhitneyu(test,c_2)

MannwhitneyuResult(statistic=9096934.5, pvalue=0.022176211299279824)

Похоже, что произошел статистически значимый сдвиг в тестовой группе(больше клиентов с подписками). 

Проверим гипотезу о равенстве средних значений при помощи бутстрапа.

In [25]:
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    sample_size = 1000
    boot_data = []

    for i in range(boot_it): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            sample_size, 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            sample_size, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) # mean() - применяем статистику
        
        
    pd_boot_data = pd.DataFrame(boot_data)

        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    ci = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )

    return {'p_value' : min(p_1, p_2) * 2}

In [26]:
get_bootstrap(c_1,c_2, statistic=np.mean)

{'p_value': 0.5776275636659932}

In [27]:
get_bootstrap(test,c_1, statistic=np.mean)

{'p_value': 0.8243757973505207}

In [28]:
get_bootstrap(test,c_2, statistic=np.mean)

{'p_value': 0.5804016426962586}

- ARPPU на клиента (test vs controls) - выросло ли значимо;

In [29]:
test_2 = users_test.query('total_revenue > 0').total_revenue
c_1_2  = users_control_1.query('total_revenue > 0').total_revenue 
c_2_2  = users_control_2.query('total_revenue > 0').total_revenue

In [30]:
mannwhitneyu(c_2_2,c_1_2) 

MannwhitneyuResult(statistic=16922.0, pvalue=0.33398146779907445)

In [31]:
mannwhitneyu(test_2,c_1_2)

MannwhitneyuResult(statistic=17281.5, pvalue=0.0002422761597780629)

In [32]:
mannwhitneyu(test_2,c_2_2)

MannwhitneyuResult(statistic=17455.5, pvalue=1.2668291046231086e-05)

In [33]:
get_bootstrap(c_1_2,c_2_2, statistic=np.median)

{'p_value': 0.2747587786493153}

In [34]:
get_bootstrap(test_2,c_1_2, statistic=np.median)

{'p_value': 4.962613165559075e-26}

In [35]:
get_bootstrap(test_2,c_2_2, statistic=np.median)

{'p_value': 3.064913374552775e-25}

In [36]:
get_bootstrap(c_1_2,c_2_2, statistic=np.mean)

{'p_value': 0.005382967410922883}

In [37]:
get_bootstrap(test_2,c_1_2, statistic=np.mean)

{'p_value': 0.07741439094303407}

In [38]:
get_bootstrap(test_2,c_2_2, statistic=np.mean) 

{'p_value': 1.5220254183245714e-17}

- частота покупок на клиента - упала или осталась на том же уровне.

In [39]:
test_3 = test_df.groupby('uid', as_index=False).agg({'paid_at':'count'})
c_1_3  = control_1_df.groupby('uid', as_index=False).agg({'paid_at':'count'})
c_2_3  = control_2_df.groupby('uid', as_index=False).agg({'paid_at':'count'})

In [40]:
mannwhitneyu(c_2_3.paid_at,c_1_3.paid_at)

MannwhitneyuResult(statistic=9247407.5, pvalue=0.8938393967482636)

In [41]:
mannwhitneyu(test_3.paid_at,c_1_3.paid_at)

MannwhitneyuResult(statistic=9249647.5, pvalue=0.011430082217034093)

In [42]:
mannwhitneyu(test_3.paid_at,c_2_3.paid_at)

MannwhitneyuResult(statistic=9093154.0, pvalue=0.017028436742515127)

In [43]:
get_bootstrap(c_2_3.paid_at,c_1_3.paid_at, statistic=np.mean)

{'p_value': 0.7798255404813423}

In [44]:
get_bootstrap(test_3.paid_at,c_1_3.paid_at,  statistic=np.mean)

{'p_value': 0.3852377891000993}

In [45]:
get_bootstrap(test_3.paid_at,c_2_3.paid_at, statistic=np.mean)

{'p_value': 0.5115088883845681}

Количество покупок пользователей не изменилось.


Теперь для пользователей, которые совершали платежи:

In [46]:
test_4 = transactions_test.groupby('uid', as_index=False).agg({'paid_at':'count'})
c_1_4  = transactions_control_1.groupby('uid', as_index=False).agg({'paid_at':'count'})
c_2_4  = transactions_control_2.groupby('uid', as_index=False).agg({'paid_at':'count'})

In [47]:
mannwhitneyu(c_2_4.paid_at,c_1_4.paid_at)

MannwhitneyuResult(statistic=18259.0, pvalue=0.8120773853913855)

In [48]:
mannwhitneyu(test_4.paid_at,c_1_4.paid_at) 

MannwhitneyuResult(statistic=14278.5, pvalue=0.8003195093470061)

In [49]:
mannwhitneyu(test_4.paid_at,c_2_4.paid_at)

MannwhitneyuResult(statistic=13675.0, pvalue=0.9744888198034231)

In [50]:
get_bootstrap(c_2_4.paid_at,c_1_4.paid_at, statistic=np.mean)

{'p_value': 0.1489173276733034}

In [51]:
get_bootstrap(test_4.paid_at,c_1_4.paid_at,  statistic=np.mean)

{'p_value': 0.7672466524679116}

In [52]:
get_bootstrap(test_4.paid_at,c_2_4.paid_at, statistic=np.mean)

{'p_value': 0.16694746976021457}

Количество покупок пользователей не изменилось.

#### Проверим, изменилось ли качество платежеспособных клиентов.

Анализ в разрезе product_type:

In [53]:
test_5 = transactions_test.groupby(['uid','product_type'], as_index=False).agg({'revenue':'sum'})
c_1_5  = transactions_control_1.groupby(['uid','product_type'], as_index=False).agg({'revenue':'sum'})
c_2_5  = transactions_control_2.groupby(['uid','product_type'], as_index=False).agg({'revenue':'sum'})

In [54]:
mannwhitneyu(c_2_5[c_2_5.product_type == 'premium_no_trial'].revenue,c_1_5[c_1_5.product_type == 'premium_no_trial'].revenue)

MannwhitneyuResult(statistic=5250.0, pvalue=0.6169279889276287)

In [55]:
mannwhitneyu(test_5[test_5.product_type == 'premium_no_trial'].revenue,c_1_5[c_1_5.product_type == 'premium_no_trial'].revenue)

MannwhitneyuResult(statistic=4949.5, pvalue=5.962668862144281e-08)

In [56]:
mannwhitneyu(test_5[test_5.product_type == 'premium_no_trial'].revenue,c_2_5[c_2_5.product_type == 'premium_no_trial'].revenue)

MannwhitneyuResult(statistic=5034.5, pvalue=1.5494296303089133e-07)

In [57]:
mannwhitneyu(c_2_5[c_2_5.product_type == 'trial_premium'].revenue,c_1_5[c_1_5.product_type == 'trial_premium'].revenue)

MannwhitneyuResult(statistic=1945.5, pvalue=0.3672329726164958)

In [58]:
mannwhitneyu(test_5[test_5.product_type   == 'trial_premium'].revenue,c_1_5[c_1_5.product_type == 'trial_premium'].revenue)

MannwhitneyuResult(statistic=2661.0, pvalue=0.0030423591708575257)

In [59]:
mannwhitneyu(test_5[test_5.product_type   == 'trial_premium'].revenue,c_2_5[c_2_5.product_type == 'trial_premium'].revenue)

MannwhitneyuResult(statistic=1937.5, pvalue=0.0666602302428119)

В типе продукта "premium_no_trial" произошли значительные изменения.

In [60]:
test_df.sort_values(['uid','paid_at']).drop_duplicates('uid').revenue.notna().mean()

0.033890436397400185

In [61]:
control_1_df.sort_values(['uid','paid_at']).drop_duplicates('uid').revenue.notna().mean()

0.044470046082949306

In [62]:
control_2_df.sort_values(['uid','paid_at']).drop_duplicates('uid').revenue.notna().mean()

0.04385553470919325

Конверсия в покупку у тестовой группы ниже, чем у обоих контрольных.

Анализ в разрезе gender:

In [63]:
test_6 = users_test.query('gender == 1').total_revenue
c_1_6  = users_control_1.query('gender == 1').total_revenue 
c_2_6  = users_control_2.query('gender == 1').total_revenue

In [64]:
get_bootstrap(c_2_6,c_1_6, statistic=np.mean)

{'p_value': 0.5708462137973656}

In [65]:
get_bootstrap(test_6,c_1_6,  statistic=np.mean)

{'p_value': 0.8605805260178212}

In [66]:
get_bootstrap(test_6,c_2_6, statistic=np.mean)

{'p_value': 0.6306029464418541}

Анализ в разрезе was_premium:

In [67]:
# все пользователи
test_7 = users_test.query('was_premium == 1').total_revenue
c_1_7  = users_control_1.query('was_premium == 1').total_revenue 
c_2_7  = users_control_2.query('was_premium == 1').total_revenue

In [68]:
get_bootstrap(c_2_7,c_1_7, statistic=np.sum)

{'p_value': 0.20224865721604912}

In [69]:
get_bootstrap(test_7,c_1_7,  statistic=np.sum)

{'p_value': 0.9599168983521966}

In [70]:
get_bootstrap(test_7,c_2_7, statistic=np.sum)

{'p_value': 0.033743937192224335}

In [71]:
# платящие пользователи
test_8 = users_test.query('was_premium == 1 and total_revenue > 0').total_revenue
c_1_8  = users_control_1.query('was_premium == 1 and total_revenue > 0').total_revenue 
c_2_8  = users_control_2.query('was_premium == 1 and total_revenue > 0').total_revenue

In [72]:
get_bootstrap(c_2_8,c_1_8, statistic=np.median)

{'p_value': 0.3666448523239304}

In [73]:
get_bootstrap(test_8,c_1_8,  statistic=np.median)

{'p_value': 4.231456307385469e-27}

In [74]:
get_bootstrap(test_8,c_2_8, statistic=np.median)

{'p_value': 1.660356707390168e-28}

In [75]:
get_bootstrap(c_2_8,c_1_8, statistic=np.sum)

{'p_value': 0.016031021520561364}

In [76]:
get_bootstrap(test_8,c_1_8,  statistic=np.sum)

{'p_value': 0.03773174885398401}

In [77]:
get_bootstrap(test_8,c_2_8, statistic=np.sum)

{'p_value': 1.6040620331538315e-18}

Анализ в разрезе is_premium:

In [78]:
test_9 = users_test.query('is_premium == 1').total_revenue
c_1_9  = users_control_1.query('is_premium == 1').total_revenue 
c_2_9  = users_control_2.query('is_premium == 1').total_revenue

In [79]:
get_bootstrap(c_2_9,c_1_9, statistic=np.mean)

{'p_value': 0.008835636279159972}

In [80]:
get_bootstrap(test_9,c_1_9,  statistic=np.mean) 

{'p_value': 0.2516935501598113}

In [81]:
get_bootstrap(test_9,c_2_9, statistic=np.mean)

{'p_value': 2.4830847693090566e-12}

### Вывод:
#### В целом видно, что изменение было эффективным. Наблюдается изменение средних, медиан и распределений самих значений между тестовой и контрольными группами по ARPU, ARPPU, а также в разрезе типов продукта, то есть уровень статистической значимости есть как для пробных подписок, так и для подписок с изменненой ценой.
#### Нововведение имеет смысл для платящих пользователей (их средний оборот на клиента вырастает из-за увеличения цены продукта), а также для клиентов, покупающих подписку без пробного периода "premium_no_trial". В разрезах gender, is_premium и was_premium (сравнивая именно по всем клиентам, а не только по заплатившим) результаты не однозначные. Часто при сравнении bootstrap показателей контрольных групп получались значимые результаты, при сравнении тестовой группы и первой контрольной группы часто можно было получить не значимые отклонения, сравнение тестовой и второй контрольной группы почти всегда показывало крайне маленький уровень значимости.